# Gewichtete Kanten
- wird benötigt um die "Entfernung" von zwei Knoten zueinander darzustellen

In [13]:
from __future__ import annotations
from dataclasses import dataclass
from edge import Edge

@dataclass
class WeightedEdge(Edge):
    weight: float
    def reversed(self) -> WeightedEdge:
        return WeightedEdge(self.v, self.u, self.weight)
    # Kanten nach Gewicht ordnen
    def __lt__(self, other: WeightedEdge) -> bool:
        return self.weight < other.weight
    def __str__(self) -> str:
        return f"{self.u} {self.weight} {self.v}"

## Gewichtete Graphen

In [14]:
from typing import TypeVar, Generic, List, Tuple
from graph import Graph
from weighted_edge import WeightedEdge

V = TypeVar('V')

class WeightedGraph(Generic[V], Graph[V]):
    def __init__(self, vertices: List[V] = []) -> None:
        self._vertices: List[V] = vertices
        self._edges: List[List[WeightedEdge]] = [[] for _ in vertices]

    def add_edge_by_indices(self, u: int, v: int, weight: float) -> None:
        edge: WeightedEdge = WeightedEdge(u, v, weight)
        self.add_edge(edge)  # Version der Elternklasse aufrufen

    def add_edge_by_vertices(self, first: V, second: V, weight: float) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v, weight)

    def neighbors_for_index_with_weights(self, index: int) -> List[Tuple[V, float]]:
        distance_tuples: List[Tuple[V, float]] = []
        for edge in self.edges_for_index(index):
            distance_tuples.append((self.vertex_at(edge.v), edge.weight))
        return distance_tuples

    def __str__(self) -> str:
        desc: str = ""
        for i in range(self.vertex_count):
            desc += f"{self.vertex_at(i)} -> {self.neighbors_for_index_with_weights(i)}\n"
        return desc


## Gewichteter Pfad

In [15]:
from typing import TypeVar, List, Optional
from weighted_graph import WeightedGraph
from weighted_edge import WeightedEdge
from Notebooks.Suche.generic_search import  PriorityQueue

V = TypeVar('V')

WeightedPath = List[WeightedEdge] # Typ-Alias für Pfade
def total_weight(wp: WeightedPath) -> float:
    return sum([e.weight for e in wp])

def mst(wg: WeightedGraph[V], start: int = 0) -> Optional[WeightedPath]:
    if start > (wg.vertex_count - 1) or start < 0:
        return None
    result: WeightedPath = []  # Enthält den finalen minimalen Spannbaum
    pq: PriorityQueue[WeightedEdge]
    visited: [bool] = [False] * wg.vertex_count  # Wo wir schon waren

    def visit(index: int):
        visited[index] = True  # Als besucht markieren
        for edge in wg.edges_for_index(index):
            # Alle von hier abgehenden Kanten zu pq hinzufügen
            if not visited[edge.v]:
                pq.push(edge)

    visit(start) # Beim ersten Knoten beginnt alles

    while not pq.empty:  # weitermachen so lange es Kanten zu verarbeiten gibt
        edge = pq.pop()
        if visited[edge.v]:
            continue # niemals erneut besuchen
        # Dies ist bisher die kürzeste Kante, also zur Lösung hinzufügen
        result.append(edge)
        visit(edge.v)
    return result

def print_weighted_path(wg: WeightedGraph, wp: WeightedPath) -> None:
    for edge in wp:
        print(f"{wg.vertex_at(edge.u)} {edge.weight}> {wg.vertex_at(edge.v)}")
    print(f"Gesamtgewicht: {total_weight(wp)}")

## Definition von gewichteten Graphen

In [16]:
from mst import mst, print_weighted_path

if __name__ == "__main__":
    city_graph2: WeightedGraph[str] = WeightedGraph(
        ["Seattle", "San Francisco", "Los Angeles", "Riverside", "Phoenix",
         "Chicago", "Boston", "New York", "Atlanta", "Miami", "Dallas",
         "Houston", "Detroit", "Philadelphia", "Washington"])

    city_graph2.add_edge_by_vertices("Seattle", "Chicago", 1737)
    city_graph2.add_edge_by_vertices("Seattle", "San Francisco", 678)
    city_graph2.add_edge_by_vertices("San Francisco", "Riverside", 386)
    city_graph2.add_edge_by_vertices("San Francisco", "Los Angeles", 348)
    city_graph2.add_edge_by_vertices("Los Angeles", "Riverside", 50)
    city_graph2.add_edge_by_vertices("Los Angeles", "Phoenix", 357)
    city_graph2.add_edge_by_vertices("Riverside", "Phoenix", 307)
    city_graph2.add_edge_by_vertices("Riverside", "Chicago", 1704)
    city_graph2.add_edge_by_vertices("Phoenix", "Dallas", 887)
    city_graph2.add_edge_by_vertices("Phoenix", "Houston", 1015)
    city_graph2.add_edge_by_vertices("Dallas", "Chicago", 805)
    city_graph2.add_edge_by_vertices("Dallas", "Atlanta", 721)
    city_graph2.add_edge_by_vertices("Dallas", "Houston", 225)
    city_graph2.add_edge_by_vertices("Houston", "Atlanta", 702)
    city_graph2.add_edge_by_vertices("Houston", "Miami", 968)
    city_graph2.add_edge_by_vertices("Atlanta", "Chicago", 588)
    city_graph2.add_edge_by_vertices("Atlanta", "Washington", 543)
    city_graph2.add_edge_by_vertices("Atlanta", "Miami", 604)
    city_graph2.add_edge_by_vertices("Miami", "Washington", 923)
    city_graph2.add_edge_by_vertices("Chicago", "Detroit", 238)
    city_graph2.add_edge_by_vertices("Detroit", "Boston", 613)
    city_graph2.add_edge_by_vertices("Detroit", "Washington", 396)
    city_graph2.add_edge_by_vertices("Detroit", "New York", 482)
    city_graph2.add_edge_by_vertices("Boston", "New York", 190)
    city_graph2.add_edge_by_vertices("New York", "Philadelphia", 81)
    city_graph2.add_edge_by_vertices("Philadelphia", "Washington", 123)
    # print(f"\n gewichtete Graphen: {city_graph2}")

    result: Optional[List[WeightedEdge]] = mst(city_graph2)
    if result is None:
        print("Keine Lösung gefunden!")
    else:
        print_weighted_path(city_graph2, result)

Seattle 678> San Francisco
San Francisco 348> Los Angeles
Los Angeles 50> Riverside
Riverside 307> Phoenix
Phoenix 887> Dallas
Dallas 225> Houston
Houston 702> Atlanta
Atlanta 543> Washington
Washington 123> Philadelphia
Philadelphia 81> New York
New York 190> Boston
Washington 396> Detroit
Detroit 238> Chicago
Atlanta 604> Miami
Gesamtgewicht: 5372
